In [1]:
import threading
import serial
import time
import matplotlib.pyplot as plt 
import numpy as np
import pyautogui
import pandas as pd
from bitstring import BitArray
from collections import deque
from tqdm import tqdm

In [2]:
# setting up lists to record into



num_channel = 4

mouse_data = []

mouse_times = []

muscle_times = []

muscle_data = []

In [3]:
#  function to record mouse positions and time of recordings
    
def measure_mouse():
        
    while run:
            
        mouse_times.append(time.time())
            
        mouse_data.append(pyautogui.position())
            
            

# Function to record raw serial data and time of recordings            
            
def measure_muscle():
        
    ser = serial.Serial('COM4', 230400)
    while run:
            
        muscle_times.append(time.time())

        muscle_data.append(self.ser.read(1))
            


In [5]:
# running recording functions semi simultaneously on two threads 

run = True
lock = threading.Lock()

t1 = threading.Thread(target=measure_muscle,)
t2 = threading.Thread(target=measure_mouse,)


t1.start(), t2.start()


time.sleep(60)

run = False

t1.join(), t2.join()


(None, None)

In [6]:
# converting bytes in muscle data to binary strings 
muscle_data = [BitArray(val).bin for val in muscle_data]

In [7]:
len(mouse_data), len(muscle_data)

(3486727, 980323)

In [8]:
# setting up mouse and muscle dataframes

mouse = pd.DataFrame({'mouse_pos':mouse_data, 'mouse_time':mouse_times})
muscle = pd.DataFrame({'muscle_data':muscle_data, 'muscle_time':muscle_times})

In [9]:
# counting frames in muscle data
count = 0
for val in muscle['muscle_data']:
   
    if int(val[0]):
        break
    else:
        pass
    count += 1
muscle = muscle[count:].reset_index(drop=True)

In [11]:
# converting binary string to list of frames and converting individual times to time ranges

frames = []
time_ranges = []
for i, val in enumerate(muscle['muscle_data']):
    if bool(int(val[0])):
        frames.append(muscle['muscle_data'][i:i+num_channel*2])
        try:
            time_ranges.append([muscle['muscle_time'][i], muscle['muscle_time'][i + num_channel * 2]])
        except:
            time_ranges.append(None)
    else:
        pass

In [12]:
# making frame vals lists
frames = [list(val) for val in frames]

In [13]:
frames = frames[1:]
time_ranges = time_ranges[1:]

In [14]:
# updating muscle data frame with time ranges and mucle frames
muscle = pd.DataFrame({'muscle_data':frames, 'muscle_time_ranges':time_ranges})

In [15]:
# peeking at data

display(muscle.head())
mouse.head()

,muscle_data,muscle_time_ranges
0,"[10000001, 00111111, 00000000, 00010011, 00000...","[1583721071.0505328, 1583721071.0515368]"
1,"[10000000, 00010110, 00000000, 00100000, 00000...","[1583721071.0515368, 1583721071.0525389]"
2,"[10000000, 00010110, 00000000, 00011111, 00000...","[1583721071.0525389, 1583721071.0525389]"
3,"[10000000, 00010110, 00000000, 00011111, 00000...","[1583721071.0525389, 1583721071.0525389]"
4,"[10000000, 00010101, 00000000, 00011111, 00000...","[1583721071.0525389, 1583721071.0525389]"


,mouse_pos,mouse_time
0,"(2044, 911)",1.583721e+09
1,"(2044, 911)",1.583721e+09
2,"(2044, 911)",1.583721e+09
3,"(2044, 911)",1.583721e+09
4,"(2044, 911)",1.583721e+09


In [16]:
# removing duplicate mouse recordings
mouse.drop_duplicates(inplace = True)

In [17]:
# down sampling mouse data to match muscle data

down_sampled_mouse = []

for val in tqdm(muscle['muscle_time_ranges']):
    mouse = mouse[mouse['mouse_time'] >= val[0]]
    
    down_sampled_mouse.append(list(mouse['mouse_pos'])[0])
        
#     except:
#         down_sampled_mouse.append([None, None])
#     for i in range(len(mouse)):
#         if mouse['mouse_time'][i] >= val[0]:
#             down_sampled_mouse.append(mouse['mouse_pos'][i])
#             break
#         else:
#             mouse.drop(i, inplace=True)
    

100%|█████████▉| 122396/122538 [27:48<00:00, 1543.73it/s]

TypeError: 'NoneType' object is not subscriptable

In [18]:
# checking lengths of muscle and mouse
len(down_sampled_mouse), len(muscle)

(122537, 122538)

In [19]:
down_sampled_mouse.append([None, None])

In [20]:
# creating final dataframe
df = muscle
df['mouse_pos'] = down_sampled_mouse

100%|█████████▉| 122396/122538 [28:01<00:00, 1543.73it/s]

In [21]:
# turning pyautogui points into two seperate columns for export

df['x_pos'] = ([val[0] for val in df['mouse_pos']])
df['y_pos'] = ([val[1] for val in df['mouse_pos']])

    
df.drop(columns = 'mouse_pos', inplace = True)


In [22]:
df.drop(columns='muscle_time_ranges', inplace=True)

In [23]:
# seperating muscle data into seperate channels
df['channel_1'] = [val[:2] for val in df['muscle_data']]
df['channel_2'] = [val[2:4] for val in df['muscle_data']]
df['channel_3'] = [val[4:6] for val in df['muscle_data']]
df['channel_4'] = [val[6:8] for val in df['muscle_data']]



In [24]:
df.drop(columns='muscle_data', inplace=True)

In [25]:
df.dropna(inplace=True)

In [26]:
df.head()

,x_pos,y_pos,channel_1,channel_2,channel_3,channel_4
0,2272.0,659.0,"[10000001, 00111111]","[00000000, 00010011]","[00000000, 00000000]","[00000000, 01010011]"
1,2268.0,661.0,"[10000000, 00010110]","[00000000, 00100000]","[00000000, 00000000]","[00000000, 01010011]"
2,2265.0,664.0,"[10000000, 00010110]","[00000000, 00011111]","[00000000, 00001100]","[00000000, 01010011]"
3,2265.0,664.0,"[10000000, 00010110]","[00000000, 00011111]","[00000000, 00010000]","[00000000, 01101111]"
4,2265.0,664.0,"[10000000, 00010101]","[00000000, 00011111]","[00000000, 00010000]","[00000000, 01101111]"


In [27]:
# making function to turn list of binary strings into integers
def raw_to_num(channel, is_first):
    if is_first:
        
        channel[0] = '0' + channel[0][1:]
  
    channel = channel[0] + channel[1]
    
    channel = int(channel, 2)
    
    return(channel)

In [28]:
# appling raw to num to all channels
df['channel_1'] = [raw_to_num(val, True) for val in df['channel_1']]
df['channel_2'] = [raw_to_num(val, False) for val in df['channel_2']]
df['channel_3'] = [raw_to_num(val, False) for val in df['channel_3']]
df['channel_4'] = [raw_to_num(val, False) for val in df['channel_4']]

In [29]:
# checking data one last time
df

,x_pos,y_pos,channel_1,channel_2,channel_3,channel_4
0,2272.0,659.0,319,19,0,83
1,2268.0,661.0,22,32,0,83
2,2265.0,664.0,22,31,12,83
3,2265.0,664.0,22,31,16,111
4,2265.0,664.0,21,31,16,111
...,...,...,...,...,...,...
122532,1577.0,1148.0,13,28,6,351
122533,1577.0,1148.0,13,28,6,351
122534,1581.0,1148.0,13,28,6,351
122535,1581.0,1148.0,13,28,6,350


In [30]:
# saving data
df.to_csv('data/raw_recording_2.csv', index=False)